# 在 Python 中使用 Ollama API



## 环境准备


在开始使用 Python 与 Ollama API 交互之前，请确保您的开发环境满足以下条件：

Python: 安装 Python 3.8 或更高版本。
pip: 确保已安装 pip，Python 的包管理工具。
ollama 库: 用于更方便地与 Ollama API 交互。安装命令如下：

```bash
pip install ollama
```

## 使用方法




In [1]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': '为什么天空是蓝色的？',
  },
])
print(response['message']['content'])

print(response.message.content)

答案很简单：

因为地球的大气层中含有大量的氮氧化物（N2O）、二氧化碳（CO2）和其他有机气体。这些气体能够吸收太阳光线中的紫外线，特别是蓝色和红色的部分。通过这个过程，大气层被分离了，导致我们看到的天空颜色变为蓝色。

但是，如果你去某些高海拔或远离城市的地区，你会发现天空颜色变成青灰色，这也不是因为大气中有更多的空气，而是因为太阳光线照射到地面时，会减少一些蓝色光线。
答案很简单：

因为地球的大气层中含有大量的氮氧化物（N2O）、二氧化碳（CO2）和其他有机气体。这些气体能够吸收太阳光线中的紫外线，特别是蓝色和红色的部分。通过这个过程，大气层被分离了，导致我们看到的天空颜色变为蓝色。

但是，如果你去某些高海拔或远离城市的地区，你会发现天空颜色变成青灰色，这也不是因为大气中有更多的空气，而是因为太阳光线照射到地面时，会减少一些蓝色光线。


## 流式响应


可以通过设置 stream=True 启用响应流，使函数调用返回一个 Python 生成器，其中每个部分都是流中的一个对象。




In [2]:
from ollama import chat

stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': '为什么天空是蓝色的？'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

这是一个古老的问题，科学家们已经有了解释。

当太阳光线从天空进入眼睛时，我们可以看到其多个颜色，包括红、橙、黄、绿、蓝、紫和红。这些颜色都来自于大气中存在的不同成分，如氮氧化物(N2O)、二氧化碳(CO2)等。

蓝色的部分主要来自于大气中的氮氧化物（N2O）。当太阳光线照射到大气时，氮氧化物与紫外线反应，产生一种名为紫外B的波长（约430-450纳米）的光。这个波长对人类眼睛最有吸引力。

在大气中，由于氮和氧成分比例的差异，大部分太阳光线被滤掉了，只剩下蓝色和紫色的部分。因此，天空看起来是蓝色的。

然而，这个解释并不是完全正确的，因为我们实际上看到的是一种叫做青绿色的颜色。这是因为大气中的其他成分，如二氧化硫、烟雾等，也会影响光线的颜色，使其变成青绿色。

所以，天空看起来蓝色的原因，是由于氮氧化物在大气中反应产生紫外B波长所致，而不是大气中存在的所有成分都直接导致了这种效果。

## 结构化输出

普通输出（Unstructured Output）:直接生成自然语言文本。适合人类阅读，但不便于程序解析或自动化处理。


结构化输出（Structured Output）:以 JSON 、 YAML 、 XML 或其他格式返回数据，使其更容易被机器解析和使用。适合 API 、自动化工作流和数据存储。


结构化输出的优势
（1）便于处理: 机器可以轻松提取特定字段，如 description 或 activity ，而无需 NLP 解析普通文本。
（2）提高可控性: 结构化格式让开发者可以精确控制模型输出，避免冗长或不可预测的回答。
（3）便于存储与分析: 结构化数据更适合存储到数据库中，方便查询和分析。




In [9]:
from pydantic import BaseModel, Field
from ollama import chat
import json

class CountryInfo(BaseModel):
    name: str = Field(..., alias="name")
    population: int = Field(..., alias="population")
    area: float = Field(..., alias="area")  


response = chat(
    model='llama3.2',
    messages=[{
        'role': 'user',
        'content': "请介绍美国的首都（capital）、人口(number)、占地面积(area)信息，并以 JSON 格式返回。"
                   
    }],
    format="json",  
    options={'temperature': 0}, 
)

response_content = response["message"]["content"]


if not response_content:
    raise ValueError("Ollama 返回的 JSON 为空")

json_response = json.loads(response_content)  
print(json_response)

friends_response = CountryInfo.model_validate(json_response)  
print(friends_response)


{'name': '华盛顿特区', 'population': 7028914, 'area': 670.2}
name='华盛顿特区' population=7028914 area=670.2


## API


### 聊天


```python
ollama.chat(model='llama3.2', messages=[{'role': 'user', 'content': '为什么天空是蓝色的？'}])
```


### 生成

```python
ollama.generate(model='llama3.2', prompt='为什么天空是蓝色的？')
```

### 本地模型列表

````python
ollama.list()
````


### 显示模型信息

```python
ollama.show('llama3.2')
```

### 创建模型

```python
modelfile='''
FROM llama3.2
SYSTEM 你是超级马里奥兄弟中的马里奥。
'''

ollama.create(model='example', modelfile=modelfile)
```


### 复制模型


```python
ollama.copy('llama3.2', 'llama3.2-copy')
```

### 删除模型

```python
ollama.delete('llama3.2-copy')
```


### 拉取模型

```python
ollama.pull('llama3.2')
```

### 推送模型

```python
ollama.push('llama3.2')
```

### 生成嵌入

```python
ollama.embeddings(model='llama3.2', prompt='为什么天空是蓝色的？')

# 批量生成embedding
ollama.embed(model='llama3.1', input=['天空是蓝色的', '草是绿色的'])
```

### 进程

```python
ollama.ps()
```

## 自定义客户端




可以通过通过 ollama 实例化 Client 或 AsyncClient 来创建自定义客户端。

可以使用以下字段创建自定义客户端：


- host: 要连接的 Ollama 主机
- timeout: 请求超时时间

### 同步客户端

使用的是同步客户端 (Client) 意味着当你调用 `client.chat()` 方法时，程序会等待该请求完成并返回结果后才会继续执行后续代码。这种方式更直观、简单，适合于编写流程较为线性、不需要处理大量并发任务的应用。






In [10]:
from ollama import Client
client = Client(
  host='http://localhost:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': '为什么天空是蓝色的？',
  },
])
print(response)


model='llama3.2' created_at='2025-03-21T12:59:53.884166Z' done=True done_reason='stop' total_duration=9174215834 load_duration=777787584 prompt_eval_count=32 prompt_eval_duration=250000000 eval_count=400 eval_duration=8145000000 message=Message(role='assistant', content='因为我们的大气层是一种透明的气体混合体，它主要由二氧化碳、氮和氧组成。这些气体在不同 altitude 的密度变化下散发出光，形成的颜色就是我们所看到的天空颜色。\n\n1.  **日射线吸收**：大气中含有大量的二氧化碳，它可以从紫外线（UV）和蓝色光谱段中吸收，这是因为在这个范围内的光波与二氧化碳的分子摩尔质量相匹配，从而引起强烈的吸收效应。\n2.  **散射效应**：当阳光照射到大气时，它会受到散射效应的影响。根据布里奇斯-马勒斯伯格定律，大气中的各个物质都有不同的吸收和反射率，这导致了不同颜色光波在进入人眼前的方式是不同的。蓝色的光波由于其短波长较小，更容易被大气中的分子吸收，从而散失。\n3.  **地面反射**：虽然大气层的蓝色是我们看到的主要颜色，但我们也需要考虑到地面的反射影响。因为当阳光照射在地面上时，会将更多的红色和橙色光波反射回天空中，而这些光波由于其长波长较大，更容易通过大气层不被吸收。这意味着在高 altitude 的环境下，我们更容易看到蓝色。这一现象是我们所看到的天空主要为蓝色的原因之一。\n\n总而言之，天空主要是蓝色的，因为它与大量二氧化碳和其他物质的吸收效应以及地面反射光子的影响有关。', images=None, tool_calls=None)


### 异步客户端

这段代码使用了异步客户端 (AsyncClient) ，并且定义了一个异步函数 chat() 。
通过await关键字，可以暂停该函数的执行直到 AsyncClient().chat() 请求完成，但在此期间不会阻塞其他操作。
这对于需要高效率处理 I/O 操作（如网络请求）或希望同时执行多个任务的应用来说非常有用。此外，使用 asyncio.run(chat()) 来运行这个异步函数。



In [11]:
import asyncio
from ollama import AsyncClient
import nest_asyncio

nest_asyncio.apply()

async def chat():
    message = {'role': 'user', 'content': '为什么天空是蓝色的？'}
    response = await AsyncClient().chat(model='llama3.2', messages=[message])
    print(response)

asyncio.run(chat())


model='llama3.2' created_at='2025-03-21T13:00:53.796113Z' done=True done_reason='stop' total_duration=4186556291 load_duration=35345375 prompt_eval_count=32 prompt_eval_duration=213000000 eval_count=196 eval_duration=3936000000 message=Message(role='assistant', content='答案：因为我们看到的是大气中小分子的散射光的颜色，主要是由碳氧化物和氮氧化物等小分子在阳光紫外线照射下吸收后发出的黄绿色光的部分。\n\n\u3000\u3000我们知道，阳光中的紫外线（UV）能量在大气中被散射出来。但是在较长波段的光（如蓝色、绿色、红色等）上，我们观察到的蓝色的颜色主要是因为大气中存在的碳氧化物和氮氧化物等小分子吸收了紫外线能量后发出的黄绿色光的部分。\n\n\u3000\u3000当我们看到的蓝色光通过大气进入我们的眼睛时，我们感觉的是这片区域的蓝色颜色，这使得我们把这个蓝色视为天空的颜色。', images=None, tool_calls=None)


设置 stream=True 修改函数以返回 Python 异步生成器：



In [12]:
import asyncio
from ollama import AsyncClient
import nest_asyncio

nest_asyncio.apply()
async def chat():
  message = {'role': 'user', 'content': '为什么天空是蓝色的？'}
  async for part in await AsyncClient().chat(model='llama3.2', messages=[message], stream=True):
    print(part['message']['content'], end='', flush=True)

asyncio.run(chat())


答案很复杂，但主要原因是我们看到的天空色彩是由太阳光线和大气层中的气体组成的。

**太阳光线**

太阳光线是一种紫外线和可见光的组合。可见光的部分包括蓝、红、黄、绿等多个颜色，分别对应不同的波长（紫外线较短，红光较长）。在太阳光线中，蓝光占比约98%。

**大气层**

当太阳光线进入大气层时，会被散射和吸收。其中，最主要的吸收剂是氮氧化物(N2O3)和臭氧(O3)，它们吸收的波长范围包括可见蓝光。因此，大气层中的这些气体使得蓝色成为我们看到最明显的颜色。

**为什么蓝空不蓝**

虽然大气层中有很多气体吸收了太阳光线，但并没有完全消去蓝光。如果观察天空，我们会看到蓝色，因为大气层中的这部分蓝光仍然被散射出来，使得我们可以看到蓝色的颜色。

**其他原因**

还有一些其他因素也会影响我们的视觉体验，例如：

* 反射：太阳光线在大气层中受到反射的作用，这可能会增加蓝色光的强度。
* 视觉心理学：人类对颜色的选择性感知和视觉表达能力也会影响我们看到的天空颜色。

总之，天空是蓝色的原因是太阳光线的组成、大气层中的气体吸收效应以及我们的视觉体验。

### 同步 & 异步客户端不同调用次数耗时对比测试

下面的这段代码分别调用同步和异步客户端重复 test_num 次问答过程，对比所需要的总时间和单次时间，用户可以更改以下的参数进行测试：

- test_messages: 测试数据
- test_num: 测试次数
- model_name: 测试模型



In [13]:
import time
import asyncio
from ollama import Client, AsyncClient
import nest_asyncio

# 应用nest_asyncio以支持Jupyter中的异步操作
nest_asyncio.apply()

# 初始化客户端
client = Client(host='http://localhost:11434')
async_client = AsyncClient(host='http://localhost:11434')


# 同步请求处理函数
def request_example(client, model_name, messages):
    start_time = time.time()
    try:
        # 同步请求返回
        response = client.chat(model=model_name, messages=messages)
    except Exception as e:
        print(f"同步请求失败: {e}")
        response = None
    end_time = time.time()
    duration = end_time - start_time
    print(f"同步请求时间: {duration}")
    return response, duration

# 异步请求处理函数
async def async_request_example(client, model_name, messages):
    start_time = time.time()
    try:
        # 异步请求返回
        response = await client.chat(model=model_name, messages=messages)
    except Exception as e:
        print(f"异步请求失败: {e}")
        response = None
    end_time = time.time()
    duration = end_time - start_time
    print(f"异步请求时间: {duration}")
    return response, duration

# 异步请求测试函数
async def async_client_test(test_num, model_name, messages):
    tasks = [asyncio.create_task(async_request_example(async_client, model_name, messages)) 
             for _ in range(test_num)]
    results= await asyncio.gather(*tasks)
    return results

# 运行同步测试
def sync_test(model_name, messages, test_num):
    total_time = 0
    for i in range(test_num):
        _, duration = request_example(client, model_name, messages)
        total_time += duration
    return total_time / test_num

# 运行异步测试
async def async_test(model_name, messages, test_num):
    start_time = time.time()
    await async_client_test(test_num, model_name, messages)
    end_time = time.time()
    return (end_time - start_time) / test_num


# 准备测试数据
test_messages = [{'role': 'user', 'content': '为什么天空是蓝色的？'}]
test_num = 10
model_name = 'llama3.2'

# 运行同步测试并输出结果
print("运行同步测试")
sync_avg_time = sync_test(model_name, test_messages, test_num)
print(f"同步测试平均时间: {sync_avg_time:.2f} 秒")

# 运行异步测试并输出结果
print("运行异步测试")
async_avg_time = asyncio.run(async_test(model_name, test_messages, test_num))
print(f"异步测试平均时间: {async_avg_time:.2f} 秒")


运行同步测试
同步请求时间: 6.623319864273071
同步请求时间: 3.462961196899414
同步请求时间: 7.9237940311431885
同步请求时间: 3.9149320125579834
同步请求时间: 7.2147581577301025
同步请求时间: 6.986761093139648
同步请求时间: 4.196196794509888
同步请求时间: 7.726733922958374
同步请求时间: 7.706302165985107
同步请求时间: 2.444819927215576
同步测试平均时间: 5.82 秒
运行异步测试
异步请求时间: 14.708611011505127
异步请求时间: 14.995337963104248
异步请求时间: 15.598981142044067
异步请求时间: 19.55816411972046
异步请求时间: 28.92905592918396
异步请求时间: 33.47827482223511
异步请求时间: 34.252699851989746
异步请求时间: 35.0913770198822
异步请求时间: 42.221561670303345
异步请求时间: 45.87859106063843
异步测试平均时间: 4.59 秒


## 错误

如果请求返回错误状态或在流式传输时检测到错误，则会引发错误。


In [16]:
import ollama

model = 'does-not-yet-exist'

try:
  ollama.chat(model)
except ollama.ResponseError as e:
  print('错误:', e.error, e.status_code)

错误: model "does-not-yet-exist" not found, try pulling it first 404
